<a href="https://colab.research.google.com/github/Manos-Mak120/Makrakis-Manos/blob/main/Copy_of_newsbomb_articles_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1].split("+")[0]
    except:
        date_str = ""
        time_str = ""

    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD


In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

**Δοκιμή του scraper**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teasers_df = pd.read_csv("/content/drive/MyDrive/articles.csv")
teasers_df.head()

,title,article_url,date,excerpt,section
0,Όλες οι πληρωμές μετά το Πάσχα - Ποιοι θα δουν...,/oikonomia/story/1643486/oles-oi-pliromes-meta...,4 εβδομάδες,Ο «χάρτης» των πληρωμών από τον e-ΕΦΚΑ και τη ...,ΟΙΚΟΝΟΜΙΑ
1,Συντάξεις Μαρτίου 2025: Πλησιάζουν οι πληρωμές...,/oikonomia/story/1628531/syntakseis-martiou-20...,20.02.2025 - 07:04,Δείτε τις αναλυτικές ημερομηνίες πληρωμής των ...,ΟΙΚΟΝΟΜΙΑ
2,Συντάξεις Μαρτίου 2025: Νωρίτερα οι πληρωμές σ...,/oikonomia/story/1627716/syntakseis-martiou-20...,17.02.2025 - 06:02,Τις αναλυτικές ημερομηνίες πληρωμής των συντάξ...,ΟΙΚΟΝΟΜΙΑ
3,Συντάξεις Οκτωβρίου: Ποιοι πληρώνονται σήμερα ...,/oikonomia/story/1587370/syntakseis-oktovriou-...,27.09.2024 - 12:26,"Ολοκληρώνονται σήμερα (27/09), οι πληρωμές των...",ΟΙΚΟΝΟΜΙΑ
4,Ολυμπιακοί Αγώνες: Από τον Φοίβο και την Αθηνά...,/kosmos/story/1570022/olympiakoi-agones-apo-to...,26.07.2024 - 11:50,Αντανακλούν την παράδοση και τον πολιτισμό της...,ΚΟΣΜΟΣ


In [ ]:
teasers_df_test = teasers_df.head(10)
teasers_df_test

,title,article_url,date,excerpt,section
0,Όλες οι πληρωμές μετά το Πάσχα - Ποιοι θα δουν...,/oikonomia/story/1643486/oles-oi-pliromes-meta...,4 εβδομάδες,Ο «χάρτης» των πληρωμών από τον e-ΕΦΚΑ και τη ...,ΟΙΚΟΝΟΜΙΑ
1,Συντάξεις Μαρτίου 2025: Πλησιάζουν οι πληρωμές...,/oikonomia/story/1628531/syntakseis-martiou-20...,20.02.2025 - 07:04,Δείτε τις αναλυτικές ημερομηνίες πληρωμής των ...,ΟΙΚΟΝΟΜΙΑ
2,Συντάξεις Μαρτίου 2025: Νωρίτερα οι πληρωμές σ...,/oikonomia/story/1627716/syntakseis-martiou-20...,17.02.2025 - 06:02,Τις αναλυτικές ημερομηνίες πληρωμής των συντάξ...,ΟΙΚΟΝΟΜΙΑ
3,Συντάξεις Οκτωβρίου: Ποιοι πληρώνονται σήμερα ...,/oikonomia/story/1587370/syntakseis-oktovriou-...,27.09.2024 - 12:26,"Ολοκληρώνονται σήμερα (27/09), οι πληρωμές των...",ΟΙΚΟΝΟΜΙΑ
4,Ολυμπιακοί Αγώνες: Από τον Φοίβο και την Αθηνά...,/kosmos/story/1570022/olympiakoi-agones-apo-to...,26.07.2024 - 11:50,Αντανακλούν την παράδοση και τον πολιτισμό της...,ΚΟΣΜΟΣ
5,Συντάξεις Ιουλίου 2024: Νωρίτερα θα δουν λεφτά...,/oikonomia/story/1559747/syntakseis-iouliou-20...,20.06.2024 - 06:06,Συντάξεις - Δείτε εδώ όλες τις ειδήσεις: Τις η...,ΟΙΚΟΝΟΜΙΑ
6,Συντάξεις Ιουλίου 2024: Πότε θα πάνε ταμείο οι...,/oikonomia/story/1555195/syntakseis-iouliou-20...,05.06.2024 - 06:14,Συντάξεις - Δείτε εδώ όλες τις ειδήσεις: Πότε ...,ΟΙΚΟΝΟΜΙΑ
7,Συντάξεις Ιουνίου 2024: Πότε θα πληρωθούν οι σ...,/oikonomia/story/1550552/syntakseis-iouniou-20...,20.05.2024 - 06:22,Συντάξεις: Οι ημερομηνίες πληρωμής ακολουθούν ...,ΟΙΚΟΝΟΜΙΑ
8,Συντάξεις Μαΐου 2024: Ξεκινούν σε λίγες μέρες ...,/oikonomia/story/1541928/syntakseis-maiou-2024...,19.04.2024 - 05:46,Την ερχόμενη εβδομάδα ξεκινούν οι πληρωμές των...,ΟΙΚΟΝΟΜΙΑ
9,Αυτοκίνητα από 300 ευρώ: Ανοιχτή μέχρι σήμερα ...,/oikonomia/story/1539836/aftokinita-apo-300-ev...,12.04.2024 - 04:56,Δημοπρασίες: Ανοικτή και σήμερα Παρασκευή 12 Α...,ΟΙΚΟΝΟΜΙΑ


In [ ]:
articles_df = scrape_newsbomb_articles_pages(teasers_df)
articles_df

Processing rows: 100%|██████████| 100/100 [03:53<00:00,  2.33s/it]


Scraping completed. Fetshed 100 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/oikonomia/story/164348...,Όλες οι πληρωμές μετά το Πάσχα - Ποιοι θα δουν...,2025-04-21,07:12:21,ΟΙΚΟΝΟΜΙΑ,Newsbomb,Ο «χάρτης» των πληρωμών από τον e-ΕΦΚΑ και τη ...,"Κατά την περίοδο από τις22 έως 25 Απριλίου, θα...",https://img.bbmd.gr/img/1260/max/82/2025/04/21...,Eurokinissi
1,https://www.newsbomb.gr/oikonomia/story/162853...,Συντάξεις Μαρτίου 2025: Πλησιάζουν οι πληρωμές...,2025-02-20,07:04:01,ΟΙΚΟΝΟΜΙΑ,Newsbomb,Δείτε τις αναλυτικές ημερομηνίες πληρωμής των ...,Η σχετική ανακοίνωση από τον ΕΦΚΑ αναφέρει πως...,https://img.bbmd.gr/img/1260/max/82/2025/02/20...,Δείτε τις ημερομηνίες\nEurokinissi
2,https://www.newsbomb.gr/oikonomia/story/162771...,Συντάξεις Μαρτίου 2025: Νωρίτερα οι πληρωμές σ...,2025-02-17,06:02:01,ΟΙΚΟΝΟΜΙΑ,Newsbomb,Τις αναλυτικές ημερομηνίες πληρωμής των συντάξ...,Ανακοίνωση από τον ΕΦΚΑ αναφέρει πως οι ημερομ...,https://img.bbmd.gr/img/1260/max/82/2025/02/11...,Δείτε τις ημερομηνίες\nEurokinissi
3,https://www.newsbomb.gr/oikonomia/story/158737...,Συντάξεις Οκτωβρίου: Ποιοι πληρώνονται σήμερα ...,2024-09-27,12:26:54,ΟΙΚΟΝΟΜΙΑ,Newsbomb,"Ολοκληρώνονται σήμερα (27/09), οι πληρωμές των...","Από προχθές, Τετάρτη 25 Σεπτεμβρίου, άρχισαν ο...",https://img.bbmd.gr/img/1260/max/82/2024/09/27...,EUROKINISSI.
4,https://www.newsbomb.gr/kosmos/story/1570022/o...,Ολυμπιακοί Αγώνες: Από τον Φοίβο και την Αθηνά...,2024-07-26,11:50:51,ΚΟΣΜΟΣ,Newsbomb,Αντανακλούν την παράδοση και τον πολιτισμό της...,"Χαριτωμένες,δημοφιλείς, αξεπέραστες ενίοτε και...",https://img.bbmd.gr/img/1260/max/82/2021/07/26...,Oι μασκότ των Ολυμπιακών Αγώνων του 2004 στην ...
...,...,...,...,...,...,...,...,...,...,...
95,https://www.newsbomb.gr/ellada/story/1376647/p...,Ποιες αργίες απομένουν από το 2022 - Όλες οι α...,2022-12-04,10:01:29,ΕΛΛΑΔΑ,Newsbomb,Ποιες είναι οι επίσημες αργίες και πώς αμείβον...,Μπήκε οΔεκέμβριοςκαι πολλοί από εμάς περιμένου...,https://img.bbmd.gr/img/1260/max/82/2022/12/04...,Όλες οι αργίες του 2022\nPixabay
96,https://www.newsbomb.gr/ellada/story/1323351/p...,Όλες οι αργίες του 2023,2022-12-04,09:54:28,ΕΛΛΑΔΑ,Γιάννης Φιλιππάκος,Ποιες είναι οι επίσημες αργίες και πώς αμείβον...,Μπήκε οΔεκέμβριοςκαι πολλοί από εμάς περιμένου...,https://img.bbmd.gr/img/1260/max/82/2022/06/13...,Όλες οι αργίες του 2022\nPixabay
97,https://www.newsbomb.gr/oikonomia/story/137661...,Συντάξεις Ιανουαρίου 2023: Νωρίτερα η καταβολή...,2022-12-04,05:15:32,ΟΙΚΟΝΟΜΙΑ,Newsbomb,Συντάξεις: Πότε θα πραγματοποιηθεί η καταβολή ...,Αλλαγέςστις ημερομηνίες καταβολής μηνιαίων κύρ...,https://img.bbmd.gr/img/1260/max/82/2022/12/04...,INTIME
98,https://www.newsbomb.gr/ellada/story/1374590/k...,Κοντογεώργης για ιδρύματα: «Θα σφίξουν τα λουρ...,2022-11-26,22:37:31,ΕΛΛΑΔΑ,Newsbomb,"O γ.γ Συντονισμού της κυβέρνησης, ο κ. Θανάσης...",OΓενικός Γραμματέας Συντονισμού της κυβέρνησης...,https://img.bbmd.gr/img/1260/max/82/2022/11/26...,


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Κατά την περίοδο από τις22 έως 25 Απριλίου, θα καταβληθούν συνολικά 1.253.742.257,83 ευρώ σε 2.530.742 δικαιούχους, στο πλαίσιο των προγραμματισμένων καταβολών του e-ΕΦΚΑ και τηςΔημόσιας Υπηρεσίας Απασχόλησης (ΔΥΠΑ). Πληρωμές e-ΕΦΚΑ Ειδικότερα, όπως αναφέρει το υπουργείο Εργασίας και Κοινωνικής Ασφάλισης σε ανακοίνωσή τουαπό τον e-ΕΦΚΑ θα γίνουν οι εξής καταβολές, στο πλαίσιο των τακτικών πληρωμών του φορέα: στις 25 Απριλίου, θα καταβληθούν 1.201.792.257,83 ευρώ σε 2.482.989 δικαιούχους για πληρωμή κύριων και επικουρικών συντάξεων Απριλίου 2025 καιαπό τις 22 Απριλίου έως τις 25 Απριλίου, θα καταβληθούν 19.000.000 ευρώ σε 750 δικαιούχους σε συνέχεια έκδοσης αποφάσεων εφάπαξ. στις 25 Απριλίου, θα καταβληθούν 1.201.792.257,83 ευρώ σε 2.482.989 δικαιούχους για πληρωμή κύριων και επικουρικών συντάξεων Απριλίου 2025 και από τις 22 Απριλίου έως τις 25 Απριλίου, θα καταβληθούν 19.000.000 ευρώ σε 750 δικαιούχους σε συνέχεια έκδοσης αποφάσεων εφάπαξ. Από τη ΔΥΠΑ θα καταβληθούν: 20.000.000 ευρώ 

In [ ]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2025/04/21/ermou-dimoskophsh.jpg?t=-VxGLONR98oXINxQX9tHxw'